In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time
from scipy.stats import invwishart, multivariate_normal
#start_time = time.time()

In [2]:
n_person  = 1000 # draws for market share integration
n_product = 10   # products
n_market  = 50   # markets
n_exp     = 5   # MC experiments
n_draw = 1000
n_mh = 1000
n_pte = 10

In [3]:
# true parameters
alpha0 = 2.0
alpha1 = 1.5
m_beta = -3.0
sd_beta = 0.5
omega0 = 1
omega1 = 0.5
m_xieta = [0, 0]
sig_xieta = np.array([[1, 0.5], [0.5, 4]])
m_x = 1
sd_x = 0.5
m_z = 1
sd_z = 0.5

In [4]:
# product characteristic x_jt and cost shifter z_jt 
x = m_x + sd_x * np.random.randn(n_product, n_market)
z = m_z + sd_z * np.random.randn(n_product, n_market)

In [5]:
# correlation b/t price component and demand shock
xieta = np.random.multivariate_normal(m_xieta, sig_xieta, (n_product, n_market))
xi = xieta[:, :, 0]
eta = xieta[:, :, 1]

In [6]:
# check xi and eta
np.cov(xi.flatten(), eta.flatten())

array([[ 0.99807716,  0.54909072],
       [ 0.54909072,  4.53783526]])

In [7]:
# Simulate p_jt
p = omega0 + omega1*z + eta

In [8]:
# Simulate consumer heterogeneity
vi = np.random.randn(n_draw)

In [9]:
# Memory allocation
s = np.zeros((n_product+1, n_market))
u = np.zeros(n_product+1)

In [10]:
# Simulate market share
u[-1] = 0   ### the last product is no purchase
for t in range(n_market):
    for i in range(n_draw): 
        for j in range(n_product):
            u[j] = alpha0 + alpha1 * x[j, t] + (m_beta + sd_beta*vi[i]) * p[j, t] + xi[j, t]
        den = 0
        for j in range(n_product+1):
            den += math.exp(u[j]-np.max(u)) # -max_u ensures that exp() <= 1 so no blowup
        # compute the logit choice probability for each choice
        for j in range(n_product+1):
            s[j, t] += math.exp(u[j]-np.max(u))/den
            
s /= n_draw
print(np.sum(s, axis=0))

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


### Estimate BLP using Bayesian

In [11]:
n_theta1  = 3 # linear parameters
n_theta2  = 1 # nonlinear parameters
n_omega = 2 # pricing equation parameters
n_obs = n_product * n_market # number of observations
tol_blp = 0.005 # tolerance level for BLP contraction mapping
rwsd = 0.05 # step size of random walk

In [12]:
# Simulate consumer heterogeneity
v = np.random.randn(n_person)

In [13]:
rho = sig_xieta[0, 1]/math.sqrt(sig_xieta[0, 0]*sig_xieta[1, 1])
# substitute accepted param for MH draws
    theta2 = theta2_a
    # pre-compute correlation
    rho = sig_xieta[0, 1]/math.sqrt(sig_xieta[0, 0]*sig_xieta[1, 1])
    # compute pllh at previously accepted theta2, theta1, omega, sig_xieta
    pllh = log_likelihood(&par, &data, &var, &pllh, 0)
    # candidate parameter value
    theta2 += rwsd*np.random.randn()
    
          //    - given theta1, compute xi
      //    - given omega, compute eta
      //    - compute jacobian
      //    - compute joint likelihood of (xi, eta)
    
    
    # calculate llh at candidate parameter value
    llh = log_likelihood(&par, &data, &var, &llh, 1)
        # acceptance probability
    acc_prob = min(0, llh - pllh)
    # accept or reject
    udraw = np.random.rand()
    if math.log(udraw) <= acc_prob:
        theta2_a = theta2
        pllh = llh
        delta_all_a = delta_all
        xi_a = xi
        eta_a = eta
    else:
        theta2 = theta2_a
        delta_all = delta_all_a
        xi = xi_a
        eta = eta_a

IndentationError: unexpected indent (<ipython-input-13-55556c88cebc>, line 3)

In [14]:
# draw theta1

### given theta2, we have delta (already computed); given omega, we have eta
### step 1. compute the mean and var of conditional dist of xi given eta
### step 2. compute the adjusted Y (delta) and X (x)
### step 3. compute the posterior dist of theta1
### step 4. draw from the distribution

def draw_theta1(delta_all, omega, sig_xieta):
    eta = P - np.dot(Z, omega)
    
    # adjust delta and X
    rho = sig_xieta[0, 1]/math.sqrt(sig_xieta[0, 0]*sig_xieta[1, 1])
    cond_m = math.sqrt(sig_xieta[0, 0]/sig_xieta[1, 1])*rho*eta
    #print(cond_m,cond_m.shape)
    cond_sd = math.sqrt((1-rho**2)*sig_xieta[0, 0])
    aY = (delta_all-cond_m)/cond_sd
    aX = X/cond_sd

    # compute posterior mean and variance of theta1
    aXtX = np.dot(aX.T, aX)
    sig_theta1 = np.linalg.inv(aXtX)
    m_theta1 = np.dot(sig_theta1, np.dot(aX.T, aY))
    theta1 = np.random.multivariate_normal(m_theta1, sig_theta1)   # should be a vector with 3 elements
    
    return theta1

# update Xtheta1: note here we have to use X, not aX
#Xtheta1 = np.dot(X, theta1)

# update xi: note here we have to use delta_all, not aY
#xi = delta_all - Xtheta1

In [15]:
# draw omega

### given theta2, we have delta (already computed); given theta1, we have xi
### step 1. compute the mean and var of conditional dist of eta given xi
### step 2. compute the adjusted Y (delta) and X (x)
### step 3. compute the posterior dist of theta1
### step 4. draw from the distribution

def draw_omega(delta_all, theta1, sig_xieta):
    xi = delta_all - np.dot(X, theta1)
    
    # adjust delta and X
    rho = sig_xieta[0, 1]/math.sqrt(sig_xieta[0, 0]*sig_xieta[1, 1])
    cond_m = math.sqrt(sig_xieta[1, 1]/sig_xieta[0, 0])*rho*xi
    cond_sd = math.sqrt((1-rho**2)*sig_xieta[1, 1])
    aP = (P-cond_m)/cond_sd
    aZ = Z/cond_sd

    # compute posterior mean and variance of theta1
    aZtZ = np.dot(aZ.T, aZ)
    sig_omega = np.linalg.inv(aZtZ)
    m_omega = np.dot(sig_omega, np.dot(aZ.T, aP))
    omega = np.random.multivariate_normal(m_omega, sig_omega)   # should be a vector with 2 elements
    
    return omega

# update Zomega: note here we have to use Z, not aZ
#Zomega = np.dot(Z, omega)

# update eta: note here we have to use P, not aP
#eta = P - Zomega

In [16]:
# draw sig_xieta

### given theta2, we have delta (already computed); given theta1, we have xi; given omega, we have eta
### step 1. compute the sample variance of (xi, eta)
### step 2. compute posterior v1 (df) and S1 (scale matrix), and transform into shape and scale
### step 3. draw from IW(shape, scale)

def draw_sig_xieta(delta_all, theta1, omega):
    # compute sample variance of (xi, eta)
    xi = delta_all - np.dot(X, theta1)
    eta = P - np.dot(Z, omega)
    xieta = np.column_stack((xi, eta))
    bar_sig = np.dot(xieta.T, xieta)# no need to divide by n_obs, as we will multiply bar_sig by n_obs
    
    # compute v1 and S1
    v1 = 2 + n_obs
    S1 = (2*np.eye(2) + bar_sig)/(2+n_obs)
    df = v1
    scale = v1*S1   # by Wikipedia

    # draw from IW(v1, S1)
    sig_xieta = invwishart.rvs(df, scale)   
    
    return sig_xieta

In [17]:
def pred_mktshare_t(deltat, theta2, t):   # t is fixed, so we only consider i and j
    pred_s = np.zeros(n_product)
    u[-1] = 0
    for i in range(n_person):
        for j in range(n_product):
            u[j] = deltat[j] + theta2*v[i]*p[j, t]
        den = 0
        for j in range(n_product+1):
            den += math.exp(u[j]-np.max(u)) # -max_u ensures that exp() <= 1 so no blowup
        # compute the logit choice probability for each choice
        for j in range(n_product):
            pred_s[j] += math.exp(u[j]-np.max(u))/den
    pred_s /= n_person
    return pred_s

In [18]:
def contraction_mapping(theta2):
    # initialize deltas for the first iteration
    #delta = np.zeros((n_product, n_market))
    
    # we can do this for each t, as markets are independent
    for t in range(n_market):
        dconv = 10000
        # initialize deltat for the first iteration
        deltat = np.log(s[:-1, t]/s[-1, t])
        obs_s = s[:-1, t]
        #print(obs_s)
        while dconv > tol_blp:
            # 1. given delta, compute predicted marekt shares
            pred_s = pred_mktshare_t(deltat, theta2, t)
            # 2. update delta
            deltan = deltat + np.log(np.divide(obs_s, pred_s))
            # 3. compute difference from previous delta
            dconv = np.max(np.abs(deltan-deltat))
            deltat = deltan
            #print("deltat mean:", np.mean(deltat))
        delta[:, t] = deltan
        #print("converged", t, dconv)  
    return delta

In [19]:
# compute the jacobian for market t
def jacobian_det(deltat, theta2, t):
    # calculate market share for each person i
    si = np.zeros((n_person, n_product))
    u[-1] = 0   ### the last product is no purchase
    for i in range(n_person): 
        for j in range(n_product):
            u[j] = deltat[j] + theta2*v[i]*p[j, t]
        den = 0
        for j in range(n_product+1):
            den += math.exp(u[j]-np.max(u)) # -max_u ensures that exp() <= 1 so no blowup
        # compute the logit choice probability for each choice
        for j in range(n_product):
            si[i, j] = math.exp(u[j]-np.max(u))/den
    
    # calculate Jacobian matrix and its invdet
    jacobian_t = np.zeros((n_product, n_product))
    for j in range(n_product):
        for k in range(n_product):
            if j == k:
                for i in range(n_person):
                    jacobian_t[j, k] += si[i, j]*(1-si[i, j])
            else:
                for i in range(n_person):
                    jacobian_t[j, k] -= si[i, j]*si[i, k]
    jacobian_t /= n_person
    det = np.linalg.det(jacobian_t)
    
    return det

In [ ]:
delta = contraction_mapping(theta2_a)
delta1 = delta[:, 0]
delta1

In [ ]:
jacobian_det(delta1, theta2_a, 0)

In [24]:
def log_likelihood(theta1, theta2, omega, sig_xieta, delta):
    
    # step 1: given theta2 (par.sd_beta in this exercise), compute delta
    delta_all = delta.flatten()
    #print(delta_all)
    
    # step 2: given delta and theta1 (previously accepted), compute xi as the residual
    Xtheta1 = np.dot(X, theta1)
    xi = delta_all - Xtheta1
    
    # step 3: given delta and omega (previously accepted), compute eta as the residual
    Zomega = np.dot(Z, omega)
    eta = P - Zomega
    
    # step 4: compute the Jacobian and likelihood
    llh = 0
    for i in range(n_obs):
        llh += multivariate_normal.logpdf([xi[i], eta[i]], mean = np.array([0, 0]), cov = sig_xieta)
    for t in range(n_market):
        deltat = delta[:, t]
        llh -= math.log(jacobian_det(deltat, theta2, t))
        
    return llh

In [21]:
# 0. initialization
delta = np.zeros((n_product, n_market))
hist_theta = np.zeros((int(n_mh/10), n_pte))
# 1. substitute
X = np.column_stack((np.ones(n_obs), x.flatten(), p.flatten()))
Z = np.column_stack((np.ones(n_obs), z.flatten()))
P = p.flatten()

In [22]:
theta2_a = np.array([0.5])
theta1 = np.array([2, 1.5, -3])
omega = np.array([1, 0.5])
sig_xieta = np.array([[1, 0.5], [0.5, 4]])

In [ ]:
pllh = log_likelihood(theta1, theta2_a, omega, sig_xieta, flg=1) 
pllh

In [141]:
start_time = time.time()

In [23]:
# MCMC loop: this version draws all alpha and betai jointly
for imh in range(1): #n_mh
    # step 1: draw theta1, omega, sig_xieta given theta2
    # given theta2_a, get delta
    delta_a = contraction_mapping(theta2_a)
    delta_all = delta_a.flatten()
    #print(delta_all)
    # Bayesian linear regression
    theta1 = draw_theta1(delta_all, omega, sig_xieta)
    print("after draw_theta1", theta1, omega, sig_xieta, theta2_a)
    omega = draw_omega(delta_all, theta1, sig_xieta)
    print("after draw_omega", theta1, omega, sig_xieta, theta2_a)
    sig_xieta = draw_sig_xieta(delta_all, theta1, omega)
    print("after draw_sig_xieta", theta1, omega, sig_xieta, theta2_a)
    
    # step 2: draw theta2 given theta1, omega, sig_xieta using MH
    # calculate pllh at previously accepted theta2_a
    pllh = log_likelihood(theta1, theta2_a, omega, sig_xieta, delta=delta_a)
    print("pllh", pllh)
    # draw candidate theta2
    theta2_c = theta2_a + rwsd*np.random.randn()
    # calculate llh at candidate theta2
    delta_c = contraction_mapping(theta2_c)
    llh = log_likelihood(theta1, theta2_c, omega, sig_xieta, delta=delta_c)
    print("llh", llh)
    # acceptance probability
    acc_prob = min(0, llh - pllh)
    # accept or reject
    udraw = np.random.rand()
    if math.log(udraw) <= acc_prob:
        theta2_a = theta2_c
        print("accept")
    else:
        print("reject")

    # display progress (every 100 iter)
    if imh % 100 == 0:
        print("\n nimh = %ld / %ld: llh = %f\n" % (imh, n_mh, pllh))
    # store mcmc draws (every 10 iter)
    if imh % 10 == 0:
        hist_theta[int(imh/10)] = np.concatenate((theta1, theta2_a, omega, sig_xieta.flatten()))
    

after draw_theta1 [ 1.3301288   1.53197819 -2.93529268] [ 1.   0.5] [[ 1.   0.5]
 [ 0.5  4. ]] [ 0.5]
after draw_omega [ 1.3301288   1.53197819 -2.93529268] [ 1.54972623  0.03150747] [[ 1.   0.5]
 [ 0.5  4. ]] [ 0.5]
after draw_sig_xieta [ 1.3301288   1.53197819 -2.93529268] [ 1.54972623  0.03150747] [[ 1.49160078  0.64890003]
 [ 0.64890003  4.24175379]] [ 0.5]
pllh 2555.69990839
llh 2563.75173507
accept

 nimh = 0 / 1000: llh = 2555.699908



In [143]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 4639.568480968475 seconds ---


In [144]:
hist_theta

array([[ 1.73463301,  1.49597946, -2.4909917 ,  0.99653677,  1.30760316,
         0.09178961,  0.96104409, -0.07463332, -0.07463332,  4.34661573],
       [ 1.78462148,  1.52393029, -2.52034542,  0.99653677,  1.22261839,
         0.16557541,  1.02238012,  0.07649231,  0.07649231,  3.90441875],
       [ 1.73957375,  1.56796414, -2.57395822,  0.99653677,  0.88050197,
         0.54960988,  1.01555044,  0.54934019,  0.54934019,  4.10946835],
       [ 1.79107012,  1.46334513, -2.55747452,  0.99653677,  1.38370495,
        -0.02957648,  1.03995323,  0.32017673,  0.32017673,  4.1885823 ],
       [ 1.8572622 ,  1.53248534, -2.6579226 ,  0.99653677,  1.44755284,
         0.09666447,  1.06198121,  0.69026811,  0.69026811,  4.67544975],
       [ 1.95918837,  1.49221214, -2.71598704,  0.99653677,  1.28678771,
         0.02544704,  1.32758368,  1.10216   ,  1.10216   ,  4.88626249],
       [ 1.99237644,  1.62686514, -2.77338064,  0.99653677,  1.31428382,
         0.1359178 ,  1.36451009,  1.2248183 

In [145]:
# compute posterior mean and sd (burn-in 0.5*n_mh)
theta = hist_theta[5:10]#[int(n_mh/20):]
m_theta = np.mean(theta, axis = 0)
sd_theta = np.std(theta, axis = 0)
print(m_theta, sd_theta)

[ 2.05612089  1.54064638 -2.81225611  0.99653677  1.19830857  0.18026444
  1.52842318  1.41806494  1.41806494  4.4254011 ] [  1.35017158e-01   1.20130342e-01   5.97444274e-02   1.11022302e-16
   9.43616849e-02   9.51315578e-02   1.51508407e-01   2.12969562e-01
   2.12969562e-01   2.44014790e-01]


In [133]:
hist_theta[9]

array([ 1.29083008,  1.64733493, -2.36242308,  0.99653677,  1.09946362,
        0.20617327,  1.00475057, -0.489418  , -0.489418  ,  3.78718813])

In [ ]:
### check: whether the calculation of  bar_sig in the sample code is the same as what we use
xi = np.random.randn(n_obs)
eta = np.random.randn(n_obs)

In [ ]:
bar_sig = np.zeros((2,2))
for i_obs in range(n_obs):
    bar_sig[0][0] += xi[i_obs]*xi[i_obs]
    bar_sig[0][1] += xi[i_obs]*eta[i_obs]
    bar_sig[1][1] += eta[i_obs]*eta[i_obs]
bar_sig

In [ ]:
try2 = np.column_stack((xi, eta))
bar_sig2 = np.dot(try2.T, try2)
bar_sig2